# Домашнее задание к лекции "Функции и работа с данными"

## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг  
- оценка 4 и меньше - средний рейтинг  
- оценка 4.5 и 5 - высокий рейтинг  

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [3]:
films = ratings.merge(movies, on='movieId', how='left')

In [4]:
total = films.pivot_table(index=['title','movieId'], values = 'rating', aggfunc='mean').reset_index()
total.head()

,title,movieId,rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",208297,3.000000
1,"""Great Performances"" Cats (1998)",51372,2.896648
2,#1 Cheerleader Camp (2010),136604,2.111111
3,#Captured (2017),183901,3.750000
4,#Female Pleasure (2018),195955,3.666667


In [7]:
def classification(data):
    if data <= 2:
        return 'низкий рейтинг'
    elif 2 < data <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [8]:
total['class'] = total['rating'].apply(classification)
total.head()

,title,movieId,rating,class
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",208297,3.000000,средний рейтинг
1,"""Great Performances"" Cats (1998)",51372,2.896648,средний рейтинг
2,#1 Cheerleader Camp (2010),136604,2.111111,средний рейтинг
3,#Captured (2017),183901,3.750000,средний рейтинг
4,#Female Pleasure (2018),195955,3.666667,средний рейтинг


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [9]:
geo_data = {

'Центр':['москва', 'тула', 'ярославль'],

'Северо-Запад':['петербург', 'псков', 'мурманск'],

'Дальний Восток':['владивосток', 'сахалин', 'хабаровск'] 
}
city=[]
for values in geo_data.values():
    for i in values:
        city.append(i)
city

['москва',
 'тула',
 'ярославль',
 'петербург',
 'псков',
 'мурманск',
 'владивосток',
 'сахалин',
 'хабаровск']

In [10]:
df = pd.read_csv('keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [11]:
def region_in_keyword(row):
    data=row['keyword'].split(' ')
    check_city = 0
    for word in data:
        if word in city:
            for items in geo_data.items():
                if word in items[1]:
                    check_city += 1
                    return items[0]
    if check_city == 0:
        return 'undefined'     

In [12]:
df['region'] = df.apply(region_in_keyword, axis=1)
df.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [13]:
df[(df['region']!='undefined')].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


## Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [14]:
films.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [15]:
years = []
for year in range(1950,2010):
    years.append(year)

In [16]:
def production_year(title):
    for year in years:
        if str(year) in title:
            return year
    return 1900

In [17]:
films['year'] = films['title'].apply(production_year)
films.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,1994
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,1993
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,1995
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,1952


In [ ]:
total = films.pivot_table(index=['year'], values = 'rating', aggfunc='mean').sort_values(by = ['rating'], ascending = False)
total.head()